In [9]:
import random
import numpy as np
import pandas as pd
import torch
import sklearn as sk
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [10]:
SEED = 42
N = 100  
random.seed(SEED)
np.random.seed(SEED)

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

indices = list(range(N))
values = np.random.rand(N).tolist() 
noise = torch.randn(N).numpy()   

In [53]:
X = pd.read_csv("Lab 05 - Regression Exam Simulation/train_dataset.csv",usecols=range(58))
y = pd.read_csv("Lab 05 - Regression Exam Simulation/train_dataset.csv",usecols=[58])
X_test = pd.read_csv("Lab 05 - Regression Exam Simulation/test_dataset.csv",usecols=range(58))

#X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42)
X_train = X.copy()
y_train = y.copy

#Search some useless columns or rows

#print(X_val.nunique())    #DROPPING COLUMNS WITH 1 VALUE, TRY ALSO WITH 2 VALUES








#fill float nan values
float_cols = X_train.select_dtypes(include=float).columns
train_means = X_train[float_cols].mean()
X_train[float_cols]= X_train[float_cols].fillna(train_means)

#float_cols = X_val.select_dtypes(include=float).columns
#X_val[float_cols]= X_val[float_cols].fillna(train_means)

float_cols = X_test.select_dtypes(include=float).columns
X_test[float_cols]= X_test[float_cols].fillna(train_means)

#transform ordinal values and fill nan
ord_cols = [f'ord_{i}' for i in range(20)]
cat_cols = [f'cat_{i}' for i in range(8)]
enc = OrdinalEncoder()

X_train.loc[:,"ord_0":"ord_19"] = enc.fit_transform(X_train.loc[:,"ord_0":"ord_19"])
train_ord_mean=X_train.loc[:,"ord_0":"ord_19"].mean()
X_train.loc[:,"ord_0":"ord_19"] = X_train.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median

#X_val.loc[:,"ord_0":"ord_19"] = enc.transform(X_val.loc[:,"ord_0":"ord_19"])
#X_val.loc[:,"ord_0":"ord_19"] = X_val.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median

X_test.loc[:,"ord_0":"ord_19"] = enc.transform(X_test.loc[:,"ord_0":"ord_19"])
X_test.loc[:,"ord_0":"ord_19"] = X_test.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median

#convert categorical values


X_train=pd.get_dummies(X_train,dummy_na=True,columns=cat_cols)
#X_val=pd.get_dummies(X_val,dummy_na=True,columns=cat_cols)
X_test=pd.get_dummies(X_test,dummy_na=True,columns=cat_cols)

#Now we can proceed with standardization
scaler = StandardScaler()
X_train.loc[:,"cont_0":"ord_19"] = scaler.fit_transform(X_train.loc[:,"cont_0":"ord_19"])
#X_val.loc[:,"cont_0":"ord_19"] = scaler.transform(X_val.loc[:,"cont_0":"ord_19"])
X_test.loc[:,"cont_0":"ord_19"] = scaler.transform(X_test.loc[:,"cont_0":"ord_19"])



C:\Users\mauri\AppData\Local\Temp\ipykernel_19956\2269132540.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train.loc[:,"ord_0":"ord_19"] = X_train.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median
C:\Users\mauri\AppData\Local\Temp\ipykernel_19956\2269132540.py:44: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test.loc[:,"ord_0":"ord_19"] = X_test.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median


In [50]:
#We can start tuning
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import GridSearchCV


Start with polynomial regression:

In [ ]:
pipe_poly = make_pipeline(PolynomialFeatures(), LinearRegression())

param_grid_poly = {
    'polynomialfeatures__degree': [1, 2],
    'linearregression__fit_intercept': [True, False]
}

grid_poly = GridSearchCV(pipe_poly, param_grid_poly, cv=5, scoring='r2', n_jobs=-1)
grid_poly.fit(X_train, y_train)
print("Best Polynomial Regression params:", grid_poly.best_params_)


KeyboardInterrupt: 